**This notebook is an exercise in the [SQL](https://www.kaggle.com/learn/intro-to-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/dansbecker/joining-data).**

---


# Introduction

[Stack Overflow](https://stackoverflow.com/) is a widely beloved question and answer site for technical questions. You'll probably use it yourself as you keep using SQL (or any programming language). 

Their data is publicly available. What cool things do you think it would be useful for?

Here's one idea:
You could set up a service that identifies the Stack Overflow users who have demonstrated expertise with a specific technology by answering related questions about it, so someone could hire those experts for in-depth help.

In this exercise, you'll write the SQL queries that might serve as the foundation for this type of service.

As usual, run the following cell to set up our feedback system before moving on.

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex6 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.
Setup Complete


Run the next cell to fetch the `stackoverflow` dataset.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Exercises

### 1) Explore the data

Before writing queries or **JOIN** clauses, you'll want to see what tables are available. 

*Hint*: Tab completion is helpful whenever you can't remember a command. Type `client.` and then hit the tab key. Don't forget the period before hitting tab.

In [3]:
# Get a list of available tables 
list_of_tables = [table.table_id for table in list(client.list_tables(dataset))] # Your code here

# Print your answer
print(list_of_tables)

# Check your answer
q_1.check()

['badges', 'comments', 'post_history', 'post_links', 'posts_answers', 'posts_moderator_nomination', 'posts_orphaned_tag_wiki', 'posts_privilege_wiki', 'posts_questions', 'posts_tag_wiki', 'posts_tag_wiki_excerpt', 'posts_wiki_placeholder', 'stackoverflow_posts', 'tags', 'users', 'votes']


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the line below.

In [4]:
#q_1.solution()

### 2) Review relevant tables

If you are interested in people who answer questions on a given topic, the `posts_answers` table is a natural place to look. Run the following cell, and look at the output.

In [5]:
# Construct a reference to the "posts_answers" table
answers_table_ref = dataset_ref.table("posts_answers")

# API request - fetch the table
answers_table = client.get_table(answers_table_ref)

# Preview the first five lines of the "posts_answers" table
client.list_rows(answers_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,14572177,None,<p>If you want to get All destop items you wil...,None,None,0,None,2013-01-28 22:11:00.227000+00:00,None,2013-01-28 22:11:00.227000+00:00,NaT,None,NaN,None,1849109,14572041,2,0,None,None
1,14572191,None,"<p>Look at this function it will work, I've co...",None,None,0,None,2013-01-28 22:12:33.897000+00:00,None,2013-01-28 22:23:02.473000+00:00,2013-01-28 22:23:02.473000+00:00,None,573032.0,None,573032,14557984,2,0,None,None
2,14572199,None,<p>Do you need to update the UI often or not?<...,None,None,0,None,2013-01-28 22:12:49.057000+00:00,None,2013-01-28 22:12:49.057000+00:00,NaT,None,NaN,None,1416035,14572118,2,0,None,None
3,14572208,None,<p>Look into using a Repeater or DataList cont...,None,None,0,None,2013-01-28 22:13:14.830000+00:00,None,2013-01-28 22:13:14.830000+00:00,NaT,None,NaN,None,358015,14572103,2,0,None,None
4,14572217,None,<p>There are some methods:</p>\n\n<ol>\n<li>Di...,None,None,0,None,2013-01-28 22:14:41.693000+00:00,None,2013-01-28 22:14:41.693000+00:00,NaT,None,NaN,None,448089,14572089,2,0,None,None


It isn't clear yet how to find users who answered questions on any given topic. But `posts_answers` has a `parent_id` column. If you are familiar with the Stack Overflow site, you might figure out that the `parent_id` is the question each post is answering.

Look at `posts_questions` using the cell below.

In [6]:
# Construct a reference to the "posts_questions" table
questions_table_ref = dataset_ref.table("posts_questions")

# API request - fetch the table
questions_table = client.get_table(questions_table_ref)

# Preview the first five lines of the "posts_questions" table
client.list_rows(questions_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,33347728,Symfony2 Assetic - how to set different direct...,<p>I'm using assetic for compiling <code>.less...,33349111.0,1,0,None,2015-10-26 14:04:13.080000+00:00,None,2015-10-27 08:46:06.270000+00:00,NaT,None,NaN,None,845978,None,1,2,symfony|assetic,256
1,33351110,Full example of container deployment with fabr...,<p>I am looking for a complete example that ex...,33357804.0,1,0,None,2015-10-26 16:43:47.483000+00:00,None,2019-01-14 17:32:34.347000+00:00,NaT,None,NaN,None,3987161,None,1,0,java|containers|kubernetes,256
2,33354185,Pelican Error: No module named typogrify.filters,"<p>I'm publishing a blog using Pelican and, fo...",33356285.0,1,0,None,2015-10-26 19:39:06.417000+00:00,None,2015-10-26 21:51:11.813000+00:00,NaT,None,NaN,None,1930564,None,1,0,python|pelican,256
3,33362029,Bitly Oauth authorization returns https://bitl...,<p>I'm trying to authorize the bitly App throu...,33362766.0,1,0,None,2015-10-27 07:22:57.270000+00:00,None,2015-10-27 08:19:33.897000+00:00,2015-10-27 07:28:18.480000+00:00,None,1092437.0,None,1092437,None,1,1,python|api|oauth|python-requests,256
4,33365872,How to create a signed url with Firebase/JWT?,<p>In the new version of Googles PHP-API-Cient...,NaN,0,0,None,2015-10-27 10:46:17.453000+00:00,None,2015-10-27 12:40:28.103000+00:00,2015-10-27 12:40:28.103000+00:00,None,4495081.0,None,2849119,None,1,3,google-api|google-cloud-storage|google-api-php...,256


Are there any fields that identify what topic or technology each question is about? If so, how could you find the IDs of users who answered questions about a specific topic?

Think about it, and then check the solution by running the code in the next cell.

In [7]:
# Check your answer (Run this code cell to receive credit!)
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
`posts_questions` has a column called `tags` which lists the topics/technologies each question is about.

`posts_answers` has a column called `parent_id` which identifies the ID of the question each answer is responding to.
`posts_answers` also has an `owner_user_id` column which specifies the ID of the user who answered the question.

You can join these two tables to:
- determine the `tags` for each answer, and then
- select the `owner_user_id` of the answers on the desired tag.

This is exactly what you will do over the next few questions.


### 3) Selecting the right questions

A lot of this data is text. 

We'll explore one last technique in this course which you can apply to this text.

A **WHERE** clause can limit your results to rows with certain text using the **LIKE** feature. For example, to select just the third row of the `pets` table from the tutorial, we could use the query in the picture below.

![](https://i.imgur.com/RccsXBr.png) 

You can also use `%` as a "wildcard" for any number of characters. So you can also get the third row with:

```
query = """
        SELECT * 
        FROM `bigquery-public-data.pet_records.pets` 
        WHERE Name LIKE '%ipl%'
        """
```

Try this yourself. Write a query that selects the `id`, `title` and `owner_user_id` columns from the `posts_questions` table. 
- Restrict the results to rows that contain the word "bigquery" in the `tags` column. 
- Include rows where there is other text in addition to the word "bigquery" (e.g., if a row has a tag "bigquery-sql", your results should include that too).

In [8]:
# Your code here
questions_query = """
                  SELECT id, title, owner_user_id
                  FROM `bigquery-public-data.stackoverflow.posts_questions`
                  WHERE tags LIKE '%bigquery%'
                  """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
questions_query_job = client.query(questions_query, job_config= safe_config)# Your code goes here

# API request - run the query, and return a pandas DataFrame
questions_results = questions_query_job.to_dataframe() # Your code goes here

# Preview results
print(questions_results.head())

# Check your answer
q_3.check()

         id                                              title  owner_user_id
0  64870863  I want to save spark JavaRDD data to bigquery ...      3020641.0
1  65163522  How can I translate an ENUM from my Avro schem...      2299087.0
2  64869110  How to see all shared queries in the "Project ...     14646421.0
3  64964489  PybigQuery: Slice a query beyond certain bytes...      8176451.0
4  65149611  Update specific BQ partition or data range wit...     13157288.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [9]:
#q_3.hint()
#q_3.solution()

### 4) Your first join
Now that you have a query to select questions on any given topic (in this case, you chose "bigquery"), you can find the answers to those questions with a **JOIN**.  

Write a query that returns the `id`, `body` and `owner_user_id` columns from the `posts_answers` table for answers to "bigquery"-related questions. 
- You should have one row in your results for each answer to a question that has "bigquery" in the tags.  
- Remember you can get the tags for a question from the `tags` column in the `posts_questions` table.

Here's a reminder of what a **JOIN** looked like in the tutorial:
```
query = """
        SELECT p.Name AS Pet_Name, o.Name AS Owner_Name
        FROM `bigquery-public-data.pet_records.pets` as p
        INNER JOIN `bigquery-public-data.pet_records.owners` as o 
            ON p.ID = o.Pet_ID
        """
```

It may be useful to scroll up and review the first several rows of the `posts_answers` and `posts_questions` tables.  

In [10]:
# Your code here
answers_query = """
                SELECT a.id, a.body, a.owner_user_id
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS q 
                INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                    ON q.id = a.parent_id
                WHERE q.tags LIKE '%bigquery%'
                """
# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
answers_query_job = client.query(answers_query, job_config= safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
answers_results = answers_query_job.to_dataframe() # Your code goes here

# Preview results
print(answers_results.head())

# Check your answer
q_4.check()

         id                                               body  owner_user_id
0  61590922  <p>Bytes processed is measured when the storag...     11467145.0
1  61596499  <p>Your query doesn't have a variable in it- i...      2259934.0
2  61612783  <pre><code>DATE_ADD( CURRENT_DATE(), INTERVAL ...     11617785.0
3  61615518  <p>Due to dependency updates in version 0.15.x...        82804.0
4  61624039  <p>This can be done using stackdriver audit lo...      5851276.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [11]:
#q_4.hint()
#q_4.solution()

### 5) Answer the question
You have the merge you need. But you want a list of users who have answered many questions... which requires more work beyond your previous result.

Write a new query that has a single row for each user who answered at least one question with a tag that includes the string "bigquery". Your results should have two columns:
- `user_id` - contains the `owner_user_id` column from the `posts_answers` table
- `number_of_answers` - contains the number of answers the user has written to "bigquery"-related questions

In [12]:
# Your code here
bigquery_experts_query = """
                            SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
                            FROM `bigquery-public-data.stackoverflow.posts_questions` AS q 
                            INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                                ON a.parent_id = q.id
                            WHERE q.tags LIKE '%bigquery%'
                            GROUP BY a.owner_user_id
                            ORDER BY number_of_answers DESC
                            
                         """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query, job_config= safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
bigquery_experts_results = bigquery_experts_query_job.to_dataframe() # Your code goes here

# Preview results
print(bigquery_experts_results.head())

# Check your answer
q_5.check()

     user_id  number_of_answers
0  5221944.0               3646
1  1144035.0               1116
2   132438.0                899
3  6253347.0                736
4  1366527.0                620


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [13]:
#q_5.hint()
#q_5.solution()

### 6) Building a more generally useful service

How could you convert what you've done to a general function a website could call on the backend to get experts on any topic?  

Think about it and then check the solution below.

In [14]:
def find(topic, client):
    query = """
           SELECT a.owner_user_id AS user_id, COUNT(1) AS num_answers
           FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
           INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
               ON q.id = a.parent_Id
           WHERE q.tags like '%{topic}%'
           GROUP BY a.owner_user_id
           """
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)      
    query_job = client.query(query, job_config=safe_config)

    result = my_query_job.to_dataframe()

    return result

In [15]:
# Check your answer (Run this code cell to receive credit!)
q_6.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

def expert_finder(topic, client):
    '''
    Returns a DataFrame with the user IDs who have written Stack Overflow answers on a topic.

    Inputs:
        topic: A string with the topic of interest
        client: A Client object that specifies the connection to the Stack Overflow dataset

    Outputs:
        results: A DataFrame with columns for user_id and number_of_answers. Follows similar logic to bigquery_experts_results shown above.
    '''
    my_query = """
               SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
               FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
               INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                   ON q.id = a.parent_Id
               WHERE q.tags like '%{topic}%'
               GROUP BY a.owner_user_id
               """
               
    # Set up the query (a real service would have good error handling for 
    # queries that scan too much data)
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)      
    my_query_job = client.query(my_query, job_config=safe_config)
    
    # API request - run the query, and return a pandas DataFrame
    results = my_query_job.to_dataframe()

    return results

```

# Congratulations!

You know all the key components to use BigQuery and SQL effectively. Your SQL skills are sufficient to unlock many of the world's largest datasets.

Want to go play with your new powers?  Kaggle has BigQuery datasets available [here](https://www.kaggle.com/datasets?fileType=bigQuery).

---




*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum/161314) to chat with other Learners.*